In [1]:
# --- IMPORTS ---
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# --- PATH SETUP ---
BASE_DIR = os.getcwd()
DATASET_PATH = os.path.join(BASE_DIR, "dataset")


TRAIN_DIR = os.path.join(DATASET_PATH, "train")
VAL_SPLIT = 0.2


# --- DATA PREPROCESSING ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=VAL_SPLIT
)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)


val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)


# --- MODEL BUILDING ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Unfreeze last convolutional blocks for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-15]:
    layer.trainable = False


x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=output)


# Compile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# --- CALLBACKS ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]


# --- TRAINING ---
EPOCHS = 20


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


# --- SAVE MODEL & WEIGHTS ---
model_save_dir = os.path.join(BASE_DIR, "saved_models")
os.makedirs(model_save_dir, exist_ok=True)


model.save(os.path.join(model_save_dir, "best_vgg_finetuned_bestrun.keras"))
model.save_weights(os.path.join(model_save_dir, "model_weights_bestrun.weights.h5"))


# Save architecture JSON
model_json = model.to_json()
with open(os.path.join(model_save_dir, "model_architecture_bestrun.json"), "w") as json_file:
    json_file.write(model_json)


print("✅ Training complete — best model and architecture saved in /saved_models/")

Found 336 images belonging to 12 classes.
Found 84 images belonging to 12 classes.
Epoch 1/20
 8/21 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.0441 - loss: 2.7560

KeyboardInterrupt: 